# Setup


In [ ]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path

import contextily as cx
import cv2
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import rasterio
from geopandas import GeoDataFrame
from shapely.geometry import Point, box

from analyser import distance_transform_scipy, road_shapefile_to_opencv_mat
from plotting import COMP_DOMAIN, export_fig, get_map_axes, plot_with_scale
from project import PRESENTATION_MEDIA_DIR

# Process Locations


In [ ]:
dir_path = Path("data/locations")
locations_file_path = dir_path / "locations.csv"
df = pd.read_csv(locations_file_path)

df["geometry"] = df.apply(lambda row: Point(row["Longitude"], row["Latitude"]), axis=1)
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")  # Setting CRS to WGS84
gdf.to_file(dir_path / "locations.shp")

gdf.plot()

offset = 0.02
offset_x = offset * (gdf.Longitude.max() - gdf.Longitude.min())
offset_y = offset * (gdf.Latitude.max() - gdf.Latitude.min())

for x, y, label in zip(
    gdf.geometry.centroid.x, gdf.geometry.centroid.y, gdf["Location"]
):
    plt.gca().text(x + offset_x, y + offset_y, label, fontsize=8, ha="center")

# Parameters


## Basemap


In [ ]:
proj_name = "RoadSectionLine"
proj_path = Path("gis/RoadSectionLine_Jul2024")
shp_file_path = proj_path / f"{proj_name}.shp"

gdf: GeoDataFrame = gpd.read_file(shp_file_path)
ax = gdf.plot()
plt.show()
x_lim = ax.get_xlim()
y_lim = ax.get_ylim()

fig, ax = plt.subplots()
ax.set_xlim(x_lim)
ax.set_ylim(y_lim)
ax = get_map_axes(ax)
cx.add_basemap(ax, crs=gdf.crs.to_string(), source=cx.providers.OpenStreetMap.Mapnik)
export_fig(fig, PRESENTATION_MEDIA_DIR / "base_map.svg")

bbox = box(
    COMP_DOMAIN.min_lon, COMP_DOMAIN.min_lat, COMP_DOMAIN.max_lon, COMP_DOMAIN.max_lat
)
bbox_gdf = gpd.GeoDataFrame([[1]], geometry=[bbox], crs="EPSG:4326")
bbox_gdf.to_crs(gdf.crs).plot(ax=ax, fc="none", edgecolor="black")

cx.add_basemap(ax, crs=gdf.crs.to_string(), source=cx.providers.OpenStreetMap.Mapnik)
export_fig(fig, PRESENTATION_MEDIA_DIR / "base_map_with_bbox.svg")

## Computation Domain


In [ ]:
gdf: GeoDataFrame = gpd.read_file(
    Path("gis/RoadSectionLine_Jul2024/RoadSectionLine.shp")
)
transformed_gdf = gdf.to_crs(crs="EPSG:4326")
points_gdf: GeoDataFrame = gpd.read_file(Path("data/locations/locations.shp"))

bbox = box(
    COMP_DOMAIN.min_lon, COMP_DOMAIN.min_lat, COMP_DOMAIN.max_lon, COMP_DOMAIN.max_lat
)
bbox_gdf = gpd.GeoDataFrame([[1]], geometry=[bbox], crs="EPSG:4326")
cropped_gdf = gpd.overlay(transformed_gdf, bbox_gdf, keep_geom_type=True)

fig, ax = plt.subplots()
ax = plot_with_scale(ax, [cropped_gdf], [{"zorder": -1}])
export_fig(fig, PRESENTATION_MEDIA_DIR / "compute_domain.svg")

fig, ax = plt.subplots()
ax = plot_with_scale(
    ax, [points_gdf, cropped_gdf], [{"color": "red", "zorder": 3}, {"zorder": -1}]
)
export_fig(fig, PRESENTATION_MEDIA_DIR / "points.svg")

## Distance Transform


In [ ]:
min_lon, max_lon = 103.76, 103.775
min_lat, max_lat = 1.303, 1.3225

bbox = box(min_lon, min_lat, max_lon, max_lat)
bbox_gdf = gpd.GeoDataFrame([[1]], geometry=[bbox], crs="EPSG:4326")
transformed_gdf = gdf.to_crs(crs="EPSG:4326")
cropped_gdf = gpd.overlay(
    transformed_gdf, bbox_gdf, how="intersection", keep_geom_type=False
)

fig, ax = plt.subplots()
ax = plot_with_scale(
    ax, [points_gdf, cropped_gdf], [{"color": "red", "zorder": 3}, {"zorder": -1}]
)
export_fig(fig, PRESENTATION_MEDIA_DIR / "vis_domain.svg")

In [ ]:
road_image = road_shapefile_to_opencv_mat(geo_data=cropped_gdf, img_width=3200)
dist = cv2.distanceTransform(cv2.bitwise_not(road_image), cv2.DIST_L2, 0)

fig, ax = plt.subplots()
ax.imshow(dist[:, :], origin="lower")
ax = get_map_axes(ax)

In [ ]:
transform, distance_transform = distance_transform_scipy(
    gdf=cropped_gdf, resolution=1e-5
)

processed_dir = proj_path / "processed"
processed_dir.mkdir(exist_ok=True)

with rasterio.open(
    proj_path / "processed" / "distance_transform.tif",
    "w",
    driver="GTiff",
    height=distance_transform.shape[0],
    width=distance_transform.shape[1],
    count=1,
    dtype=distance_transform.dtype,
    crs=cropped_gdf.crs,
    transform=transform,
) as dst:
    dst.write(distance_transform, 1)

plt.imshow(distance_transform)
plt.colorbar()
plt.show()